In [1]:
"""
This module contains the workflow which combines pre-converged single-impurity calculations to a larger impurity
"""

from __future__ import absolute_import
from __future__ import print_function
from aiida.engine import WorkChain, if_, ToContext, calcfunction
from aiida.orm import load_node, Dict, WorkChainNode, Int, RemoteData
from aiida_kkr.calculations import KkrCalculation, KkrimpCalculation
from aiida_kkr.workflows import kkr_imp_sub_wc, kkr_flex_wc, kkr_imp_wc
from aiida_kkr.tools.combine_imps import (create_combined_imp_info_cf, combine_potentials_cf,
                                          get_zimp, get_host_structure, get_nspin_and_pot, combine_settings_ldau)
from aiida_kkr.tools.save_output_nodes import create_out_dict_node


__copyright__ = (u"Copyright (c), 2020, Forschungszentrum Jülich GmbH, "
                 "IAS-1/PGI-1, Germany. All rights reserved.")
__license__ = "MIT license, see LICENSE.txt file"
__version__ = "0.3.1"
__contributors__ = (u"Philipp Rüßmann")


class combine_imps_wc(WorkChain):
    

    _workflowversion = __version__
    _wf_default = {'jij_run': False,
                    }

    @classmethod
    def get_wf_defaults(cls, silent=False):

        if not silent:
            print('Version of workflow: {}'.format(cls._workflowversion))
        return cls._wf_default


    @classmethod
    def define(cls, spec):
        """
        Defines the outline of the workflow
        """

        # take define from AiiDA base class and extend it then
        super(combine_imps_wc, cls).define(spec)

        # expose these inputs from sub-workflows
        spec.expose_inputs(kkr_imp_sub_wc, namespace='scf', include=('kkrimp', 'options', 'wf_parameters',))
        spec.expose_inputs(kkr_flex_wc,
                           namespace='host_gf',
                           include=('kkr', 'options', 'params_kkr_overwrite',), # expose only those port which are not set automatically
                           namespace_options={'required': False, 'populate_defaults': False}, # this makes sure the kkr code input is not needed if gf_host_remote is provided and the entire namespace is omitted
                          )

        # define the inputs of the workflow

        # mandatory inputs
        spec.input("impurity1_output_node", required=True, valid_type=Dict, #TODO make validator for input node to make sure it is the output of kkr_imp_wc
                   help="""jjj""")
        
        spec.input("impurity2_output_node", required=True, valid_type=Dict,
                   help="nnnodd"
        
        spec.input("offset_imp2", valid_type=Dict, required=True,
                   help="""ure.""")
        spec.input("wf_parameters_overwrite", valid_type=Dict, required=False,
                    help="To add or edit wf_parameters in scf namespace and add run optioins, if needed")
        spec.input("gf_host_remote", valid_type=RemoteData, required=False, #TODO add validator that makes sure this is not given together with the host_gf sub-workflow namespace
                   help="""RemoteData node of pre-calculated host Green function (i.e. with kkr_flex_wc).
If given then the writeout step of the host GF is omitted.""")


        # structure of the workflow
        spec.outline(
            cls.start,                      # initialize workflow (set things in context and some consistency checks)
            cls.create_big_cluster,         # combine imp clusters of the two imps
            if_(cls.need_gf_run)(           # check if GF was given in input and can be reused
                cls.run_gf_writeout),       # write out the host GF
            cls.check_host_gf,
            cls.create_big_potential,       # combine preconverged potentials to big one
            cls.update_kkrimp_params,       # update wf_parameters of scf namespace from _wf_defaults
            cls.run_kkrimp_scf,             # run the kkrimp_sub workflow to converge the host-imp startpot
            if_(cls.run_jij)(
            cls.run_jij_step),                    # run jij step
            cls.return_results)             # check if the calculation was successful and return the result nodes


        # define the possible exit codes
        spec.exit_code(999, 'ERROR_SOMETHING_WENT_WRONG',
            message="ERROR: take a look")
        spec.exit_code(900, 'ERROR_HOST_STRUCTURES_INCONSISTENT',
            message="Host structures of impurity 1 and impurity 2 are not identical!")
        spec.exit_code(800, 'ERROR_INPUT_NOT_SINGLE_IMP_CALC',
            message="Impurity input is not a single impurity calculation.")
        spec.exit_code(850, 'ERROR_INPLANE_NEIGHBOR_TOO_SMALL',
            message="i_neighbor_inplane needs to be positive and bigger than 0 for in-plane neighbors")
        spec.exit_code(950, 'ERROR_INCONSISTENT_NSPIN_VALUES',
            message="The impurity calculations have different NSPIN values")
        spec.exit_code(700, 'ERROR_HOST_GF_CALC_FAILED',
            message="The writeout of the host GF failed")

        #TODO to fix this create_combined_imp_info_cf need to take the different layers into account
        # when the difference vector and the neighbors are created

        # define the outputs of the workflow
        spec.output('workflow_info')
        spec.output('last_calc_output_parameters')
        spec.output('last_potential')
        spec.output('last_calc_remote')
        spec.output('remote_data_gf')


    def start(self): # pylint: disable=inconsistent-return-statements
        """
        prepare context and do some consistency checks
        """
        inputs = self.inputs
        print('This is debug line to check the debug', inputs)
        message = 'INFO: started combine_imps_wc workflow version {}'.format(self._workflowversion)
        self.report(message)
        if 'wf_parameters_overwrite' in inputs:
            self.ctx.wf_parameters_overwrite= self.inputs.wf_parameters_overwrite
        self.ctx.imp1 = self.get_imp_node_from_input(iimp=1)
        self.ctx.imp2 = self.get_imp_node_from_input(iimp=2)
        
        # find and compare host structures for the two imps to make sure the impurities are consistent
        host_structure1 = get_host_structure(self.ctx.imp1)
        host_structure2 = get_host_structure(self.ctx.imp2)
        #TODO this can be relaxed to make sure the same structure is used even if it is not the same node
        if host_structure1.uuid != host_structure2.uuid:
            self.report("host structures inconsistent")
            return self.exit_codes.ERROR_HOST_STRUCTURES_INCONSISTENT # pylint: disable=maybe-no-member

        # save host structure in context
        self.ctx.host_structure = host_structure1

        # settings for offset between imps
        self.ctx.i_neighbor_inplane = self.inputs.offset_imp2['index']
        
        # preserve the inputs from scf namespace to context
        self.ctx.scf_kkrimp = self.inputs.scf.kkrimp
        if 'options' in self.inputs.scf:
            self.ctx.scf_options = self.inputs.scf.options
        if 'wf_parameters' in self.inputs.scf:
            self.ctx.scf_wf_parameters = self.inputs.scf.wf_parameters
        # Add some run option here
        self.ctx.jij_option = False

        # TODO: PRESERVE THE INPUTS FROM host_gf NAMESPACE TO CONTEXT
        # TODO: ALSO EDIT THE RUN_GF_WRITEOUT() FOR THIS CORRESPONDING CHANGES


SyntaxError: invalid syntax (<ipython-input-1-cdf17314ed06>, line 64)